In [1]:
import torch
# assert '.'.join(torch.__version__.split('.')[:2]) == '1.4'
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import numpy as np

import torch.nn.functional as F  # useful stateless functions


In [2]:
from utils import PCamDataset

In [3]:
from three_layers_convnet import ThreeLayerConvNet

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [5]:
def check_accuracy_part34(loader, model):
    #if loader.dataset.train:
    #    print('Checking accuracy on validation set')
    #else:
    #    print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [6]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            #x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            #y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            print(scores, y)
            #print(scores, scores.size(), torch.squeeze(scores), y)
            loss = F.nll_loss(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [7]:
train_dataset = PCamDataset(csv_file="train_labels.csv")
val_dataset = PCamDataset(csv_file="dev_labels.csv")

In [8]:
batch_size = 2

loader_train = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              shuffle=True)
loader_val = DataLoader(val_dataset, 
                              batch_size=batch_size, 
                              shuffle=True)

In [9]:
learning_rate = 3e-3
channel_1 = 96
channel_2 = 48

model = ThreeLayerConvNet(in_channel=3, 
                          channel_1=channel_1,
                          channel_2=channel_2,
                          num_classes=1)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

c:\users\julienveronvialard\anaconda3\envs\cs231n\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.],
        [0.]], grad_fn=<SigmoidBackward>) tensor([[1],
        [0]])


RuntimeError: multi-target not supported at c:\a\w\1\s\tmp_conda_3.7_110509\conda\conda-bld\pytorch_1544094576194\work\aten\src\thnn\generic/ClassNLLCriterion.c:21